# Simple Example

In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

Let's load our quickstart and translate to spanish

In [2]:
from pathlib import Path

In [3]:
config_folder = Path("configs")
md_file = Path("docs/quickstart.md")

In [4]:
from gpt_translate.translate import Translator

In [5]:
t = Translator(config_folder, language="es", max_chunk_tokens=400)

Model args: {'model': 'gpt-3.5-turbo', 'temperature': 0.7}


In [6]:
# es_quickstart = t.translate_file(md_file)

# Api Example

In [7]:
from gpt_translate.translate import translate_splitted_md
from gpt_translate.loader import split_markdown, remove_markdown_comments, MDPage

In [8]:
with open(md_file, "r") as f:
    md_content = f.read()

model_args = dict(
    model="gpt-4",
    temperature= 0.7,
    )

prompt_template = t.prompt_template

In [11]:
md_content = remove_markdown_comments(md_content)

In [18]:
md_page = MDPage.create("quickstart", md_content)

In [19]:
md_page.header

---
description: W&B Quickstart.
displayed_sidebar: default
---
import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';

In [20]:
chunks = split_markdown(md_content)

In [21]:
chunks

["---\ndescription: W&B Quickstart.\ndisplayed_sidebar: default\n---\n\nimport Tabs from '@theme/Tabs';\nimport TabItem from '@theme/TabItem';",
 '# Quickstart\n\nInstall W&B and start tracking your machine learning experiments in minutes.',
 '## 1. Create an account and install W&B\nBefore you get started, make sure you create an account and install W&B:\n\n1. [Sign up](https://wandb.ai/site) for a free account at [https://wandb.ai/site](https://wandb.ai/site) and then login to your wandb account.  \n2. Install the wandb library on your machine in a Python 3 environment using [`pip`](https://pypi.org/project/wandb/).  \n\n\nThe following code snippets demonstrate how to install and log into W&B using the W&B CLI and Python Library:\n\n<Tabs\n  defaultValue="notebook"\n  values={[\n    {label: \'Notebook\', value: \'notebook\'},\n    {label: \'Command Line\', value: \'cli\'},\n  ]}>\n  <TabItem value="cli">\n\nInstall the CLI and Python library for interacting with the Weights and Bias

In [22]:
print(chunks[0])

---
description: W&B Quickstart.
displayed_sidebar: default
---

import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';


In [23]:
translated_file = translate_splitted_md(chunks,
                                        prompt_template,
                                        max_chunk_tokens=2000, 
                                        **model_args)

INFO:root:Calling OpenAI with {'model': 'gpt-4', 'temperature': 0.7}
Translating chunk: 

---
description: W&B Quickstart.
displayed_sidebar: default
---

import Tabs from '@theme/Tabs';
i...


Packing chunk 0 with 33 tokens
Packing chunk 1 with 17 tokens
Packing chunk 2 with 251 tokens
Packing chunk 3 with 190 tokens
Packing chunk 4 with 178 tokens
Packing chunk 5 with 380 tokens
Packing chunk 6 with 199 tokens
Packing chunk 7 with 244 tokens
>> Translating 1492 tokens (last chunk)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:OpenAI response: ---
description: Inicio rápido de W&B.
displayed_sidebar: default
---

import Tabs from '@theme/Tabs...


Function translate_splitted_md took 198.1675 seconds to execute.


In [24]:
translated_md_page = MDPage.create("quickstart_es", translated_file)